### TP Transformer BERT : Classification de texte avec BERT

`Mélissa Ben Saada`
`Amy EBEN SANG KOTTA`

#### **Objectif** :
Utiliser trois modèles BERT différents pour effectuer une tâche de classification de texte (analyse de sentiment) et comparer leurs performances à l'aide de métriques.

---

#### **1. Importation des bibliothèques**

In [4]:
# Importation des bibliothèques nécessaires

from transformers import pipeline
from sklearn.metrics import classification_report
import pandas as pd

2024-11-21 11:48:50.374655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### **2. Chargement des modèles et des tokenizers**
Sélectionnons trois modèles BERT différents :
1. **`distilbert-base-uncased-finetuned-sst-2-english`** (une version allégée de BERT)
2. **`distilbert-base-uncased`** (une version allégée de BERT)
3. **`roberta-base`** (basé sur BERT mais avec des optimisations)

In [10]:
# Chargement des modèles et création des pipelines
models = {
    "distilbert-base-uncased-finetuned-sst-2-english": pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english"),
    "distilbert-base-uncased": pipeline("sentiment-analysis", model="distilbert-base-uncased"),
    "roberta-base": pipeline("sentiment-analysis", model="roberta-base")
}

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exa

#### **3. Création d'un pipeline pour la classification de texte**
Testons chaque modèle avec un ensemble d'exemples.

In [11]:
# Phrases d'exemple
examples = [
    "I love this product! It works perfectly and is exactly what I needed.",
    "This is the worst experience I've ever had. Completely disappointed.",
    "The quality is decent for the price, but I expected better."
]

# Analyser les phrases avec chaque modèle
results = {}
for model_name, classifier in models.items():
    results[model_name] = [classifier(text)[0] for text in examples]

# Affichage des résultats
for model_name, output in results.items():
    print(f"\nResults for {model_name}:")
    for i, result in enumerate(output):
        print(f"Example {i+1}: {result}")



Results for distilbert-base-uncased-finetuned-sst-2-english:
Example 1: {'label': 'POSITIVE', 'score': 0.9998830556869507}
Example 2: {'label': 'NEGATIVE', 'score': 0.9998016953468323}
Example 3: {'label': 'POSITIVE', 'score': 0.5213127136230469}

Results for distilbert-base-uncased:
Example 1: {'label': 'LABEL_0', 'score': 0.5148850679397583}
Example 2: {'label': 'LABEL_0', 'score': 0.5238295197486877}
Example 3: {'label': 'LABEL_0', 'score': 0.5196446776390076}

Results for roberta-base:
Example 1: {'label': 'LABEL_1', 'score': 0.5098674297332764}
Example 2: {'label': 'LABEL_1', 'score': 0.512851893901825}
Example 3: {'label': 'LABEL_1', 'score': 0.511315643787384}


#### **4. Évaluation des performances avec des métriques**
Créons un jeu de données de test avec des labels (0 pour négatif, 1 pour positif) et comparons les prédictions des modèles.

In [12]:
# Données étiquetées
test_data = [
    {"text": "I love this product! It works perfectly and is exactly what I needed.", "label": 1},
    {"text": "This is the worst experience I've ever had. Completely disappointed.", "label": 0},
    {"text": "The quality is decent for the price, but I expected better.", "label": 1},
]

# Évaluer chaque modèle
for model_name, classifier in models.items():
    print(f"\nEvaluating {model_name}...")
    y_true = [data["label"] for data in test_data]
    y_pred = []
    for data in test_data:
        prediction = classifier(data["text"])[0]["label"]
        y_pred.append(1 if prediction == "POSITIVE" else 0)
    
    # Calcul des métriques
    print(classification_report(y_true, y_pred, target_names=["NEGATIVE", "POSITIVE"]))


Evaluating distilbert-base-uncased-finetuned-sst-2-english...
              precision    recall  f1-score   support

    NEGATIVE       1.00      1.00      1.00         1
    POSITIVE       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3


Evaluating distilbert-base-uncased...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

    NEGATIVE       0.33      1.00      0.50         1
    POSITIVE       0.00      0.00      0.00         2

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3


Evaluating roberta-base...
              precision    recall  f1-score   support

    NEGATIVE       0.33      1.00      0.50         1
    POSITIVE       0.00      0.00      0.00         2

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

### **1. distilbert-base-uncased-finetuned-sst-2-english**

Le modèle **`distilbert-base-uncased-finetuned-sst-2-english`** est une version allégée de BERT, appelée **DistilBERT**, fine-tunée spécifiquement pour l’analyse de sentiments. Ce fine-tuning a été effectué sur le dataset **SST-2 (Stanford Sentiment Treebank version 2)**, un ensemble de données contenant des critiques de films annotées comme **positives** ou **négatives**. Par exemple :
- "A wonderful little film about love." → **POSITIVE**
- "An absolute mess." → **NEGATIVE**

**DistilBERT** a été préentraîné sur des données textuelles massives comme **Wikipedia** et le **BookCorpus**, deux sources riches en contenu généraliste. Ce modèle est particulièrement adapté pour des tâches comme l’analyse de sentiments sans nécessiter de modification supplémentaire, grâce à son fine-tuning sur SST-2.

---

### **2. distilbert-base-uncased**

Le modèle **`distilbert-base-uncased`** est une version légère et optimisée de BERT. Il ne fait pas la distinction entre les majuscules et les minuscules (**uncased**) et est conçu pour être plus rapide et moins coûteux en ressources tout en conservant une grande partie des performances de BERT. Contrairement à la version fine-tunée sur SST-2, celui-ci n’est pas spécialisé dans une tâche précise.

Ce modèle a été préentraîné sur des données textuelles massives comme **Wikipedia** et le **BookCorpus**, ce qui lui permet de comprendre le langage de manière générale. Cependant, pour des tâches spécifiques comme l’analyse de sentiments, ce modèle nécessite un **fine-tuning** supplémentaire pour apprendre les nuances des données spécifiques.

---

### **3. roberta-base**

Le modèle **`roberta-base`** est une version optimisée et plus robuste de BERT, développée par Facebook AI. Contrairement à BERT, RoBERTa a été préentraîné sur un corpus de données beaucoup plus grand, comprenant :
- **Wikipedia**
- **BookCorpus**
- **Common Crawl** : un ensemble massif de données textuelles collectées sur le Web.
- **OpenWebText** : des textes issus de liens partagés sur Reddit.
- **Stories** : des textes narratifs extraits de sites comme Gutenberg.

RoBERTa se distingue par plusieurs optimisations, comme l’utilisation de séquences plus longues et la suppression des tokens de segmentation. Cela le rend plus performant pour des tâches variées, mais il nécessite un **fine-tuning** pour atteindre une performance optimale sur des tâches spécifiques, comme l’analyse de sentiments.

---

### **Résumé des sources de données**
Tous ces modèles partagent des données communes pour leur préentraînement, principalement **Wikipedia** et **BookCorpus**, ce qui leur permet de développer une compréhension générale du langage. Les données additionnelles et les fine-tunings, comme SST-2 pour `distilbert-base-uncased-finetuned-sst-2-english`, les rendent spécialisés pour des tâches précises comme l’analyse de sentiments.